In [34]:
#needed packages
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
from Functions import extractor

#options for displaying dataframe
pd.options.display.max_seq_items = 20000
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

In [35]:
#Load the data
a=open("/Users/klaus/Documents/Jarvis Data/Data/data (1).json")
data=json.load(a)
df_json=json_normalize(data,sep="->")
df_json.shape
df_json.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/klaus/Documents/Jarvis Data/Data/data (1).json'

In [6]:
# one hot encode the inputs of technologies 
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df = df_json.join(pd.DataFrame(mlb.fit_transform(df_json.pop('technologies')),
                               columns=mlb.classes_,
                               index=df_json.index))

print(list(df.columns))
df.head()

KeyError: 'technologies'

In [5]:
#prepare columns on which the tables can be merged
%store -r stat_webs
import re
a=[]
for i in range(len(df["website_url"])):
    a.append(re.split(r'[;,./:]', str(df["website_url"][i])))
b=[]
for i in range(len(a)):
    n=2
    try:
        if len(a[i][-1])!=0:
            word=[a[i][-1]]
            while a[i][-n]!="http" and a[i][-n]!="www" and a[i][-n]!="https" and a[i][-n]!="on" and a[i][-(n+1)]!="de" and a[i][-n]+a[i][-n-1]!="":
                word.append(a[i][-n].lower())
                n+=1
            b.append(".".join(reversed(word)))
        else:
            word=[a[i][-2]]
            while a[i][-(n+1)]!="http" and a[i][-(n+1)]!="www" and a[i][-(n+1)]!="https" and a[i][-(n+1)]!="on" and a[i][-(n+1)]!="de"  and a[i][-(n+1)]+a[i][-(n+2)]!="":
                word.append(a[i][-n-1].lower())
                n+=1
            b.append(".".join(reversed(word)))
    except:
        b.append("")
    

In [30]:
#a
#b


['trustyou.com',
 'pathfindr.dev',
 'healthcaters.de',
 'kollabo.com',
 'kasuria.io',
 'govradar.net',
 'jiffsy.co',
 'ekidz.eu',
 'aklimate.co.uk',
 'syntinels.com',
 'neuron.world',
 'supplyet.com',
 'cash-on-ledger.com',
 'uleagz.com',
 'neuro-flash.com',
 'lhotse-analytics.com',
 'accessia.io',
 'letslama.com',
 'build38.com',
 'lentvio.io',
 'medicalytix.ai',
 'mutumutu.com',
 'plenti.app',
 'tasteindex.com',
 'napo.pet',
 'sentian.ai',
 'getpaid.io',
 'smino.ch',
 'lotaro.de',
 'videosupport.io',
 'storypod.com',
 'voxalyze.com',
 'valyria.de',
 'lupiter.com',
 'prodlane.io',
 'cledara.com',
 'impactbytes.io',
 'finway.de',
 'insureq.de',
 'prevency.com',
 'vialytics.de',
 'urbyo.com',
 'neotradebank.com',
 'traxo.com',
 'foodcircle.com',
 'orangery.io',
 'charter-space.com',
 'blockchain-helix.com',
 'amzscale.net',
 'telosbrands.com',
 'skash.com',
 'dabbel.eu',
 'datakili.com',
 'linkapital.com.br',
 'eyeson.com',
 'gravitydata.co',
 'companiions.com',
 'bling.de',
 'summ-ai.c

In [6]:
df["Website"]=b


In [7]:
df[["name","Website"]].head()

,name,Website
0,TrustYou,trustyou.com
1,pathfindr ApS,pathfindr.dev
2,HealthCaters,healthcaters.de
3,Kollabo,kollabo.com
4,KASURIA GmbH,kasuria.io


In [9]:
#remove errors from list of dependent variables
json_error=open("/Users/klaus/Documents/Jarvis Data/Data/error.json")
error=pd.DataFrame(json.load(json_error))
error.rename({0:"Website"},axis="columns",inplace=True)
stat_webs_we=stat_webs[~stat_webs["Website"].isin(error["Website"])]
stat_webs_we.head()

,Name,Website,Status
0,TrustYou,trustyou.com,Leads
1,Pathfindr,pathfindr.dev,Leads
2,healthcaters,healthcaters.de,Leads
4,Kollabo,kollabo.com,Rejected
5,Kasuria,kasuria.io,Leads


In [11]:
#b.index("long.com")
#stat_webs_we["Website"].loc["getneuro.ai"]
#df.loc[12]

stat_webs_we["Website"].head()

0       trustyou.com
1      pathfindr.dev
2    healthcaters.de
4        kollabo.com
5         kasuria.io
Name: Website, dtype: object

In [12]:

len([x for x in list(b) if x not in list(stat_webs_we["Website"])])

125

In [14]:
#Merging the dependent variable into the Dataframe
model_data=pd.merge(df, stat_webs_we, on="Website", how="left")
model_data[["name","Website","website_url","Status"]].isna().sum()

name             0
Website          0
website_url      0
Status         125
dtype: int64

In [15]:
model_data[model_data['Website'] == 'getneuro.ai'].index
stat_webs.iloc[10:20]

,Name,Website,Status
10,Pronary,pronary.de,Prospect
11,SYNTINELS,syntinels.com,Leads
12,Neuron,neuron.world,Prospect
13,Fractive Technologies,fractive.de,Prospect
14,SupplyET Supply Engine Technologies,supplyet.com,Watch/Hold/Reconsider
15,CASH ON LEDGER,cash-on-ledger.com,Watch/Hold/Reconsider
16,UKIXX,uleagz.com,Prospect
17,CyberDesk,long-llama.super.site,Rejected
18,SMAWK,aservment.com,Rejected
19,LabV Intelligent Solutions,labv.io,Rejected


In [16]:
# deleting the na from Status
model_data.dropna(subset=["Status"],inplace=True)
model_data[["name","Website","website_url","Status"]].isna().sum()

name           0
Website        0
website_url    0
Status         0
dtype: int64

In [18]:
pip install pandas_profiling

     |████████████████████████████████| 268 kB 465 kB/s eta 0:00:01
     |████████████████████████████████| 667 kB 618 kB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 8.4 MB/s  eta 0:00:01
     |████████████████████████████████| 9.6 MB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 11.4 MB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 102 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 233 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 5.5 MB/s eta 0:00:011
     |████████████████████████████████| 296 kB 14.9 MB/s eta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27084 sha256=283f65b6f22dbcd24e43a3c4c7f902ecfc85f13d5594575a883ae22ae924f524
  Stored in directory: /U

In [1]:

import pandas_profiling as pp
profile=pp.ProfileReport(model_data, minimal=True)
profile.to_file(output_file="<DealroomDataAnalysis>.html")
profile

ModuleNotFoundError: No module named 'pandas_profiling'

The following will be a brief analysis of the fields provided in the dataset. 
The goal is to eliminate fields that provide insufficient completeness of data and seek methods to fill empty fields for those that are left.

The Profiling Report (DealroomDataAnalysis.html) is used as main reference.

Kriteria on which columns will be dropped:
1.no informative value (ex.: id)
2.According to missing values or zeros, which are synonymous in several columns, where it doesnt make sense to fill them
(20-30% of missing values will be kept. Anything above can be included if it provides better results)
(some of the dropped values are dictionaries which should be unpacked and added, task for later stage)

The columns dropped are accordingly:
1.Website, Name, kpi_summary->last_update_date, kpi_summary->id,created_utc,website_url,url,tagline,path,deleted,id
2.traffic, exits, exits->total, exits->items, traffic->top_countries, traffic->sources,funding->items, fundings->total, investors->total, traffic->annual_growth,
investors->total, investors->items, team->items, kpi_summary->values_sources,kpi_summary->valuations,kpi_summary->values,kpi_summary->currency, trading_multiple,
employee_12_month_growth,employee_6_month_growth,app_12_months_growth,app_6_months_growth,app_3_months_growth,similarweb_12_month_growth,similarweb_6_month_growth,
similarweb_3_month_growth,last_funding_source,total_funding_source,launch_month,delivery_method,service_industries,corporate_industries



In [38]:
model_data.columns

NameError: name 'model_data' is not defined

In [48]:
# drop columns that are redundant
cols=["Website","Name","kpi_summary->last_update_date","kpi_summary->id","last_updated","website_url","url","tagline","path","deleted","id","traffic","exits->total","exits->items","traffic->top_countries","traffic->sources","fundings->items","fundings->total","investors->total","traffic->annual_growth","investors->total","investors->items","team->items","kpi_summary->values_sources","kpi_summary->valuations","kpi_summary->values","kpi_summary->currency","trading_multiple","employee_12_months_growth_unique","employee_6_months_growth_unique","app_12_months_growth_unique","app_6_months_growth_unique","app_3_months_growth_unique","similarweb_12_months_growth_unique","similarweb_6_months_growth_unique","similarweb_3_months_growth_unique","last_funding_source","total_funding_source","launch_month","delivery_method","service_industries","corporate_industries"]     
model_data.drop(columns=cols,inplace=True)
model_data.drop_duplicates(subset="name", keep="first",inplace=True)

#with open('clean_data.json', 'w') as f:
    #json.dump(model_data, f)

In [58]:
#Change the dependent variable to binary 
#model_data.Status.unique()
model_data["Status"]=np.where((model_data["Status"]=="Rejected") | (model_data["Status"]=="To be rejected"), 0,1)

#Change values where it makes sense
model_data["traffic->visitors"].replace([None], 0, inplace=True)


In [59]:
#sin,cos transformation of date
#transformation into datetime type
dateadj_data=model_data.copy(deep=True)
dateadj_data["last_updated_utc"] =  pd.to_datetime(model_data["last_updated_utc"])
dateadj_data["kpi_summary->last_update_date_utc"] =  pd.to_datetime(model_data["kpi_summary->last_update_date_utc"])
dateadj_data["created_utc"] =  pd.to_datetime(model_data["created_utc"])
dateadj_data["launch_year"] =  pd.to_datetime(model_data["launch_year"],format="%Y").dt.year

#new columns with cos and sin data for month. year will be same. dropping old ones
dateadj_data["last_updated_month_cos"]=np.cos(2 * np.pi * dateadj_data["last_updated_utc"].dt.month/12.0)
dateadj_data["last_updated_month_sin"]=np.sin(2 * np.pi * dateadj_data["last_updated_utc"].dt.month/12.0)
dateadj_data["last_updated_year"]=dateadj_data["last_updated_utc"].dt.year

dateadj_data["kpi_summary->last_update_date_utc_month_cos"]=np.cos(2 * np.pi * dateadj_data["kpi_summary->last_update_date_utc"].dt.month/12.0)
dateadj_data["kpi_summary->last_update_date_utc_month_sin"]=np.sin(2 * np.pi * dateadj_data["kpi_summary->last_update_date_utc"].dt.month/12.0)
dateadj_data["kpi_summary->last_update_date_utc_year"]=dateadj_data["kpi_summary->last_update_date_utc"].dt.year

dateadj_data["created_utc_month_cos"]=np.cos(2 * np.pi * dateadj_data["created_utc"].dt.month/12.0)
dateadj_data["created_utc_month_sin"]=np.sin(2 * np.pi * dateadj_data["created_utc"].dt.month/12.0)
dateadj_data["created_utc_year"]=dateadj_data["created_utc"].dt.year

dateadj_data.drop(columns=["last_updated_utc","kpi_summary->last_update_date_utc","created_utc"],inplace=True)

In [ ]:
#Exporting Data
dateadj_data.to_csv("Dateadjusted_data.csv",index=False) 
model_data.to_csv("Clean_Data.csv",index=False) 

In [60]:
dateadj_data.columns

Index(['name', 'type', 'about', 'employees', 'employees_latest', 'growth_stage', 'traffic_summary', 'launch_year', 'has_promising_founder', 'has_strong_founder', 'has_super_founder', 'total_funding', 'last_funding', 'company_status', 'employee_3_months_growth_unique', 'job_openings', 'team->total', 'traffic->visitors', '3d technology', 'artificial intelligence', 'augmented reality', 'autonomous & sensor tech', 'big data', 'blockchain', 'computer vision', 'connected device', 'deep learning', 'deep tech', 'hardware', 'iot internetofthings', 'machine learning', 'mobile app', 'nanotech', 'natural language processing', 'quantum technologies', 'recognition technology', 'virtual reality', 'Status', 'last_updated_month_cos', 'last_updated_month_sin', 'last_updated_year', 'kpi_summary->last_update_date_utc_month_cos', 'kpi_summary->last_update_date_utc_month_sin', 'kpi_summary->last_update_date_utc_year', 'created_utc_month_cos', 'created_utc_month_sin', 'created_utc_year'], dtype='object')

In [76]:
from fastai.tabular.all import *
cat_names = ["name",'type', "growth_stage", "has_promising_founder", "has_strong_founder", "has_super_founder",'company_status', ]
cont_names = ['employees', 'employees_latest', 'traffic_summary','total_funding', 'last_funding','employee_3_months_growth_unique','job_openings','team->total', 'traffic->visitors', 'last_updated_month_cos', 'last_updated_month_sin', 'last_updated_year', 'kpi_summary->last_update_date_utc_month_cos', 'kpi_summary->last_update_date_utc_month_sin', 'kpi_summary->last_update_date_utc_year', 'created_utc_month_cos', 'created_utc_month_sin', 'created_utc_year']
procs = [Categorify, FillMissing, Normalize]
path = "/notebooks/API Calls and Datacleaning/Data/Clean_Data.csv"
to = TabularDataLoaders.from_df(dateadj_data, path,procs=procs, cat_names=cat_names, cont_names=cont_names, y_names="Status")

TypeError: 'Categorical' with dtype category does not support reduction 'median'

zsh:1: no matches found: pymongo[srv]
Note: you may need to restart the kernel to use updated packages.


/Users/klaus/Documents/Jarvis/Functions.py:46: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df_1=json_normalize(json_dic,sep="->")
/Users/klaus/Documents/Jarvis/Functions.py:46: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df_1=json_normalize(json_dic,sep="->")


""
0
1
2
3
4
5
6
7
8
9
